# Import general dependencies

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os
import cv2
import glob


%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# Import Data

In [ ]:
def train_impath():
    labels = pd.read_csv('../input/labels.csv')
    lab = labels.iloc[:,0].tolist()

    data_dir = []
    for l in lab:
        t_dir = "../input/train/"+l+".jpg"
        data_dir.append(t_dir)
    return data_dir

In [ ]:
def test_impath():
    labels = pd.read_csv('../input/sample_submission.csv')
    lab = labels.iloc[:,0].tolist()

    data_dir = []
    for l in lab:
        t_dir = "../input/test/"+l+".jpg"
        data_dir.append(t_dir)
    return data_dir

In [ ]:
train_dir = train_impath()
test_dir = test_impath()

In [ ]:
# data: str-> name of the dataSet
def im_read(data_dir):
    df = []
    for path in data_dir[0:100]:
        img = cv2.imread(path,0)
        img = cv2.resize(img,(80,80)).reshape(80,80,1)
        df.append(img.tolist())
    return np.array(df, dtype=float).reshape(-1,80,80,1)/255

In [ ]:
t = im_read(train_dir) #test dataSet
ts = im_read(test_dir) #train dataSet

# Data preparation

In [ ]:
lebels = pd.read_csv('../input/labels.csv')

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
le = LabelEncoder()
enc = OneHotEncoder()

In [ ]:
#Labels
leb = lebels.iloc[:,1].value_counts().index.sort_values()
leb = [[l] for l in leb]

In [ ]:
#Lebel Encoding
le = le.fit(leb)
lebels.breed = le.transform(lebels.breed)

In [ ]:
#OneHotEncoding
leb_enc = le.transform(leb)
leb_enc = [[l] for l in leb_enc]

In [ ]:
breed = lebels.breed.tolist()
breed = [[b] for b in breed]

In [ ]:
enc = enc.fit(leb_enc)
breed_enc = enc.transform(breed).toarray()

In [ ]:
X = t.copy()

In [ ]:
del t

In [ ]:
Y = breed_enc[0:100]

In [ ]:
X_ts = ts.copy()

In [ ]:
del ts

In [ ]:
#import packages
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPool2D,Flatten, LeakyReLU

In [ ]:
# Set the CNN model 
# my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out

model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='linear', input_shape = (80,80,1)))
model.add(LeakyReLU(alpha=.001))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))
# model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
#                  activation ='linear', input_shape = (200,200,1)))
# model.add(LeakyReLU(alpha=.001))
# model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
# model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation = "linear"))
model.add(LeakyReLU(alpha=.001))
model.add(Dropout(0.5))
model.add(Dense(256, activation = "linear"))
model.add(LeakyReLU(alpha=.001))
#model.add(Dropout(0.5))
model.add(Dense(120, activation = "relu"))

In [ ]:
from keras import optimizers
opt = optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
model.compile(optimizer = opt , loss = 'mse', metrics=['mae','accuracy'])

In [ ]:
model.fit(X, Y, epochs=1, batch_size=100,validation_split = 0.2)

In [ ]:
Y_ts = model.predict(X_ts)

In [ ]:
labels = pd.read_csv('../input/labels.csv')
columns = labels.iloc[:,1].value_counts().index.sort_values()

In [ ]:
ans = pd.DataFrame(Y_ts, columns = columns)

In [ ]:
sub = pd.read_csv('../input/sample_submission.csv')

In [ ]:
ids = pd.DataFrame(sub.id)

In [ ]:
result = pd.concat([ids, ans], axis=1, sort=False)

In [ ]:
result.to_csv('sub1.csv',index=False)